# Albedo Radiative Forcing Demo
Interactive-style walkthrough: choose a surface, apply an albedo perturbation, compute top-of-atmosphere (TOA) forcing, and validate against a benchmark sensitivity.

In [ ]:
import sys, pathlib
import pandas as pd

# Ensure repo root is on the path when running from notebooks/
repo_root = pathlib.Path.cwd().parent
if str(repo_root) not in sys.path:
    sys.path.insert(0, str(repo_root))

from src import albedo, model, validation

# Inspect available surface classes
pd.DataFrame.from_dict(albedo.SURFACE_LIBRARY, orient='index')

## Single-scenario calculation
Edit the parameters below to explore a different surface or perturbation.

In [ ]:
# User-editable parameters
surface = 'vegetation'      # pick from the table above
albedo_delta = -0.02         # additive change (negative = darkening)
area_fraction = 0.5          # fraction of Earth affected
anchor = 'typical'           # 'typical', 'min', or 'max' baseline

scenario, forcing_result = model.albedo_pipeline(
    surface_type=surface,
    albedo_delta=albedo_delta,
    anchor=anchor,
    area_fraction=area_fraction,
)
scenario, forcing_result

In [ ]:
# Validate against IPCC-style benchmark sensitivity (-340 W/m^2 per unit Δα, scaled by area)
validation_result = validation.validate_forcing_result(forcing_result)
validation_result

## Sensitivity sweep
Explore forcing across multiple perturbations for one surface type.

In [ ]:
surface = 'urban'\narea_fraction = 0.2\ndeltas = [-0.05, -0.02, 0.0, 0.02, 0.05]\nrows = []\nfor d in deltas:\n    scen, force = model.albedo_pipeline(surface, albedo_delta=d, area_fraction=area_fraction)\n    val = validation.validate_forcing_result(force)\n    rows.append({\n        'delta_alpha': d,\n        'forcing_W_m2': force.radiative_forcing_w_m2,\n        'within_expected': val.within_range,\n        'expected_low': val.expected_range_w_m2[0],\n        'expected_high': val.expected_range_w_m2[1],\n    })\npd.DataFrame(rows)\n